# Explorando os dados de vendas de carros do eBay

Antes de mais nada, para nossa análise, levemos em conta que ao contrário dos veículos novos, os carros usados já sofreram alguns graus de desgaste.

Você pode ser um comprador de primeira viagem procurando seu primeiro carro usado ou um entusiasta rastreando aquele negócio de melhor valor. Sites de carros simplificam a experiência de compra com listagens detalhadas e pesquisáveis, avaliações de carros, guias de compradores e muito mais. Esses sites não apenas ajudam os compradores a encontrar carros, eles são um excelente recurso para avaliar as opções de compra em termos de valor potencial pelo dinheiro pago.

Exploraremos um conjunto de dados de carros usados do eBay Kleinanzeigen, uma seção de classificados do site alemão do eBay. Nosso objetivo é obter alguns insights sobre a natureza e a popularidade dos carros listados e, em seguida, entender quais carros valem mais a pena para compradores preocupados com o preço.

## Visão geral

Das listagens do eBay Kleinanzeigen, observamos que mesmo um veículo com baixa quilometragem pode suportar mais do que seu quinhão de desgaste, impactando negativamente o valor. Portanto, é essencial criar uma métrica que combine preço, danos não reparados e quilometragem para determinar o valor geral de uma compra potencial.

## Dados


Os dados originais foram simplificados para facilitar o trabalho. Você pode ter acesso ao conjunto de dados original por meio deste [link](https://data.world/data-society/used-cars-data).

Visão geral das 20 colunas do conjunto de dados:

`dateCrawled:`- Quando este anúncio foi buscado pela primeira vez. Todos os valores são obtidos a partir desta data.

`name`:- Nome do carro.

`seller`:- Se o vendedor é particular ou revendedor.

`offerType`:- O tipo de listagem

`price`:- O preço no anúncio para vender o carro.

`abtest`:- Se a listagem está incluída em um teste A/B.

`VehicleType`: - O tipo de veículo.

`yearOfRegistration` - O ano em que o carro foi registrado pela primeira vez.

`gearbox`:- O tipo de transmissão.

`powerPS`:- A potência do carro em PS.

`model`:- O nome do modelo do carro.

`kilometer`:- Quantos quilômetros o carro percorreu.

`monthOfRegistration`:- O mês em que o carro foi registrado pela primeira vez.

`fuelType`:- Que tipo de combustível o carro usa.

`brand`:- A marca do carro.

`notRepairedDamage`:- Se o carro tiver um dano que ainda não foi reparado.

`dateCreated`: - A data em que a listagem do eBay foi criada.

`nrOfPictures`:- O número de fotos no anúncio.

`postalCode`:- O código postal da localização do veículo.

`lastSeenOnline`:- Quando o rastreador viu este anúncio pela última vez online.


## Importando Bibliotecas

In [1]:
# importar bibliotecas
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import math

%matplotlib inline 

In [2]:
# ler dados
autos = pd.read_csv("autos.csv", encoding = 'Latin-1')
print("O arquivo contém", len(autos.columns), "colunas e ", len(autos.index), "linhas" )

O arquivo contém 20 colunas e  50000 linhas


## Examinando os dados

In [3]:
# estatísticas
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [4]:
# usamos o método 'df' para ver o início e o fim dos dados no momento
autos

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,privat,Angebot,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,2016-03-27 00:00:00,0,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,privat,Angebot,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,2016-03-28 00:00:00,0,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,privat,Angebot,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,2016-04-02 00:00:00,0,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,privat,Angebot,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,2016-03-08 00:00:00,0,35683,2016-04-05 16:45:07


In [5]:
# examinar 10 linhas aleatórias
autos.sample(10).sort_index()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
16168,2016-03-13 08:47:44,Opel_Zafira_1.8_16V,privat,Angebot,"$2,400",control,bus,2003,manuell,125,zafira,"150,000km",3,benzin,opel,nein,2016-03-13 00:00:00,0,17489,2016-03-24 05:16:55
16908,2016-03-21 21:50:35,Ford_Probe_24V,privat,Angebot,"$1,600",control,coupe,1995,manuell,162,andere,"150,000km",4,benzin,ford,nein,2016-03-21 00:00:00,0,78224,2016-04-07 02:46:33
18566,2016-03-28 19:55:41,BMW_525d_Aut.,privat,Angebot,"$10,000",control,limousine,2006,automatik,177,5er,"150,000km",5,diesel,bmw,nein,2016-03-28 00:00:00,0,93426,2016-04-07 02:17:54
22390,2016-03-12 11:45:55,Mazda_mpv,privat,Angebot,"$2,700",test,NaN,2017,manuell,136,andere,"150,000km",7,diesel,mazda,nein,2016-03-12 00:00:00,0,17491,2016-03-29 18:46:55
26278,2016-03-27 13:37:58,autoverkaufen,privat,Angebot,$400,test,limousine,2003,manuell,82,NaN,"150,000km",3,diesel,hyundai,nein,2016-03-27 00:00:00,0,42857,2016-04-07 09:16:09
28279,2016-03-26 14:59:33,Skoda_Octavia_1.6,privat,Angebot,"$5,200",control,limousine,2009,manuell,102,octavia,"80,000km",9,benzin,skoda,nein,2016-03-26 00:00:00,0,17309,2016-04-06 02:46:33
30738,2016-03-29 23:57:32,Vw_t3_1.6td_Pritsche,privat,Angebot,"$1,200",control,bus,1988,manuell,69,transporter,"5,000km",0,NaN,volkswagen,ja,2016-03-29 00:00:00,0,66798,2016-04-06 21:47:38
31442,2016-04-03 16:58:28,Seat_Toledo_GT_2l__mit_Tuev_und_Klima,privat,Angebot,$499,control,limousine,1992,manuell,115,toledo,"150,000km",4,benzin,seat,ja,2016-04-03 00:00:00,0,58239,2016-04-05 16:46:42
32030,2016-04-04 09:55:56,Ford_Transit_330_K_TDCi_EUROLINE,privat,Angebot,"$18,900",control,kombi,2013,manuell,140,transit,"40,000km",3,diesel,ford,nein,2016-04-04 00:00:00,0,96231,2016-04-04 10:44:26
33998,2016-03-18 19:45:40,Audi_A3_3.2_quattro_Sportback__DSG__S_tronic,privat,Angebot,"$10,499",control,kombi,2006,automatik,250,a3,"150,000km",6,benzin,audi,nein,2016-03-18 00:00:00,0,96126,2016-04-05 22:47:00


Podemos então observar:

5000 linhas em 20 colunas.

Os dados em 5 colunas são do tipo int64.

Os dados em 15 colunas são do tipo objeto (string).

As colunas `vehicleType`, `gearbox`, `model`, `fuelType`, `notRepairDamage` têm dados nulos.

As informações nas colunas `dataCrawled`, `dataCreated` e `lastSeen` têm o formato "AAAA-MM-DD HH:MM:SS" e são armazenadas como string.

As informações na coluna `name` têm sublinhados, barras invertidas e outros símbolos e, em geral, não são muito compreensíveis.

O preço é em dólares americanos e armazenado como string.

Os nomes das colunas não são fáceis de entender. Veremos que não basta apenas renomeá-los, mas convertê-los também.

In [6]:
#nomes originais
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [7]:
#instalando o stringcase
%pip install stringcase

You should consider upgrading via the '/dataquest/system/env/python3/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
#importando o stringcase e o snakecase da biblioteca
import stringcase as stringcase
from stringcase import snakecase

In [9]:
#transformando colunas camelcase em colunas snakecase
snakecase_col = []    # cria uma lista vazia de nomes de colunas
    
for col in autos.columns: # usar um for
    col = snakecase(col)
    snakecase_col.append(col)
    
autos.columns = snakecase_col 

#renomeando algumas colunas com novos valores
renomear_colunas = [
    ['abtest', 'ab_test'],
    ['year_of_registration','registration_year'],
    ['month_of_registration','registration_month'],
    ['not_repaired_damage','unrepaired_damage'],
    ['date_created','ad_created'],
    ['power_p_s','power_ps']
]

for col_ant, col_nov in renomear_colunas:
    autos.rename({col_ant: col_nov},axis=1,inplace=True)

In [10]:
# nomes das novas colunas

autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [11]:
autos.describe(include='all')


,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-16 21:50:53,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


### Observações

* a coluna `name` contém informações ilegíveis, por isso precisa ser trabalhada
* as colunas `vehicle_type`, `fuel_type`, `unrepaired_damage` têm valores vazios
* alguns dados contêm termos alemães 
* as colunas `price` e `odometer` devem ser numéricas, mas estão armazenadas como strings

## Limpando os Dados

Descartaremos algumas colunas que não nos serão úteis:

In [12]:
# salvando os dados em um novo arquivo
autos.to_csv('autos_step_1.csv', index = False)
# ler o novo arquivo
carros_1 = pd.read_csv("autos_step_1.csv", encoding = 'Latin-1')

In [13]:
carros_1.drop(['seller','offer_type','nr_of_pictures', 'postal_code','registration_month' ],axis=1,inplace=True)

In [14]:
carros_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date_crawled       50000 non-null  object
 1   name               50000 non-null  object
 2   price              50000 non-null  object
 3   ab_test            50000 non-null  object
 4   vehicle_type       44905 non-null  object
 5   registration_year  50000 non-null  int64 
 6   gearbox            47320 non-null  object
 7   power_ps           50000 non-null  int64 
 8   model              47242 non-null  object
 9   odometer           50000 non-null  object
 10  fuel_type          45518 non-null  object
 11  brand              50000 non-null  object
 12  unrepaired_damage  40171 non-null  object
 13  ad_created         50000 non-null  object
 14  last_seen          50000 non-null  object
dtypes: int64(2), object(13)
memory usage: 5.7+ MB


Agora há 15 colunas 

#### Valores negativos

In [15]:
carros_1[["vehicle_type", "fuel_type", "unrepaired_damage"]] = carros_1[["vehicle_type", "fuel_type", "unrepaired_damage"]].fillna('unknown')

In [16]:
carros_1['fuel_type'].unique()


array(['lpg', 'benzin', 'diesel', 'unknown', 'cng', 'hybrid', 'elektro',
       'andere'], dtype=object)

In [17]:
carros_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date_crawled       50000 non-null  object
 1   name               50000 non-null  object
 2   price              50000 non-null  object
 3   ab_test            50000 non-null  object
 4   vehicle_type       50000 non-null  object
 5   registration_year  50000 non-null  int64 
 6   gearbox            47320 non-null  object
 7   power_ps           50000 non-null  int64 
 8   model              47242 non-null  object
 9   odometer           50000 non-null  object
 10  fuel_type          50000 non-null  object
 11  brand              50000 non-null  object
 12  unrepaired_damage  50000 non-null  object
 13  ad_created         50000 non-null  object
 14  last_seen          50000 non-null  object
dtypes: int64(2), object(13)
memory usage: 5.7+ MB


Ora, mais duas colunas têm valor vazio. 
Como já preenchemos várias colunas com valores, faremos o mesmo, e não alteraremos o código acima.

In [18]:
carros_1[["gearbox", "model"]] = carros_1[["gearbox", "model"]].fillna('unknown')

In [19]:
carros_1['gearbox'].unique()

array(['manuell', 'automatik', 'unknown'], dtype=object)

In [20]:
carros_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date_crawled       50000 non-null  object
 1   name               50000 non-null  object
 2   price              50000 non-null  object
 3   ab_test            50000 non-null  object
 4   vehicle_type       50000 non-null  object
 5   registration_year  50000 non-null  int64 
 6   gearbox            50000 non-null  object
 7   power_ps           50000 non-null  int64 
 8   model              50000 non-null  object
 9   odometer           50000 non-null  object
 10  fuel_type          50000 non-null  object
 11  brand              50000 non-null  object
 12  unrepaired_damage  50000 non-null  object
 13  ad_created         50000 non-null  object
 14  last_seen          50000 non-null  object
dtypes: int64(2), object(13)
memory usage: 5.7+ MB


Finalmente, não há nenhum valor vazio. Vamos para o próximo passo:
* traduzir os significados para o inglês

In [21]:
carros_1["unrepaired_damage"] = carros_1["unrepaired_damage"].str.replace('nein','Não')\
.str.replace('ja', 'Sim').str.replace('unknown', 'Desconhecido')
un_damage_counts = carros_1['unrepaired_damage'].value_counts()
print(un_damage_counts)

Não             35232
Desconhecido     9829
Sim              4939
Name: unrepaired_damage, dtype: int64


In [22]:
carros_1["vehicle_type"] = carros_1["vehicle_type"].str.replace('kleinwagen','small_car').str.replace('limousine', 'sedan').str.replace('cabrio', 'convertible')\
    .str.replace('andere', 'other').str.replace('kombi','estate_wagon')
vehicle_type_counts = carros_1['vehicle_type'].value_counts()
print(vehicle_type_counts)

sedan           12859
small_car       10822
estate_wagon     9127
unknown          5095
bus              4093
convertible      3061
coupe            2537
suv              1986
other             420
Name: vehicle_type, dtype: int64


In [23]:
carros_1["gearbox"] = carros_1["gearbox"].str.replace('manuell','manual').str.replace('automatik', 'automatico')

In [24]:
carros_1['gearbox'].unique()

array(['manual', 'automatico', 'unknown'], dtype=object)

## Substituindo Nomes

In [25]:
carros_1['brand']= carros_1['brand'].str.title()

In [26]:
carros_1['brand'].unique()

array(['Peugeot', 'Bmw', 'Volkswagen', 'Smart', 'Ford', 'Chrysler',
       'Seat', 'Renault', 'Mercedes_Benz', 'Audi', 'Sonstige_Autos',
       'Opel', 'Mazda', 'Porsche', 'Mini', 'Toyota', 'Dacia', 'Nissan',
       'Jeep', 'Saab', 'Volvo', 'Mitsubishi', 'Jaguar', 'Fiat', 'Skoda',
       'Subaru', 'Kia', 'Citroen', 'Chevrolet', 'Hyundai', 'Honda',
       'Daewoo', 'Suzuki', 'Trabant', 'Land_Rover', 'Alfa_Romeo', 'Lada',
       'Rover', 'Daihatsu', 'Lancia'], dtype=object)

In [27]:
# Substituição dos nomes
carros_1["brand"] = carros_1["brand"].str.replace('Bmw','BMW').str.replace('Seat', 'SEAT')\
    .str.replace('Mini', 'MINI').str.replace('Saab','SAAB').str.replace('Mercedes_Benz', 'Mercedes Benz')\
    .str.replace('Sonstige_Autos', 'Sonstige Autos').str.replace('Land_Rover', 'Land Rover').str.replace('Alfa_Romeo', 'Alfa Romeo')

In [28]:
# Verifica com que frequência cada marca aparece nos dados
n_marcas = carros_1['brand'].value_counts()
print(n_marcas)

Volkswagen        10687
Opel               5461
BMW                5429
Mercedes Benz      4734
Audi               4283
Ford               3479
Renault            2404
Peugeot            1456
Fiat               1308
SEAT                941
Skoda               786
Mazda               757
Nissan              754
Citroen             701
Smart               701
Toyota              617
Sonstige Autos      546
Hyundai             488
Volvo               457
MINI                424
Mitsubishi          406
Honda               399
Kia                 356
Alfa Romeo          329
Porsche             294
Suzuki              293
Chevrolet           283
Chrysler            181
Dacia               129
Daihatsu            128
Jeep                110
Subaru              109
Land Rover           99
SAAB                 80
Daewoo               79
Trabant              78
Jaguar               77
Rover                69
Lancia               57
Lada                 31
Name: brand, dtype: int64


In [29]:
print(" A marca mais popular é a Volkswagen (", n_marcas.max(),
      "cars), a menos popular é a Lada (", n_marcas.min(),"cars).\n")

 A marca mais popular é a Volkswagen ( 10687 cars), a menos popular é a Lada ( 31 cars).



As colunas `preço` e `odômetro` devem ser numéricas, mas são armazenadas como strings, portanto, devem ser limpas e convertidas para o formato numérico.

In [30]:
# Coluna de preços
n_preco = carros_1['price'].value_counts()
print(n_preco)

$0          1421
$500         781
$1,500       734
$2,500       643
$1,200       639
            ... 
$7,240         1
$30,899        1
$9,280         1
$310           1
$145,000       1
Name: price, Length: 2357, dtype: int64


In [31]:
# Removendo caracteres
carros_1["price_$"] = carros_1["price"].str.replace("$","").str.replace(",", "")

In [32]:
carros_1["price_$"].str.isdigit().unique()

array([ True])

In [33]:
# Odômetro
n_odo = carros_1['odometer'].value_counts()
print(n_odo)

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64


In [34]:
# Removendo  'km' e ','.
carros_1["odometer_km"] = carros_1["odometer"].str.replace("km","").str.replace(",", "")

In [35]:
carros_1["odometer_km"].str.isdigit().unique()

array([ True])

#### Convertendo os dados para o formato numérico

In [36]:
carros_1["price_$"] = carros_1["price_$"].astype(int)
carros_1["odometer_km"] = carros_1["odometer_km"].astype(int)

In [37]:
print("Tipo da coluna preço :", carros_1["price_$"].dtype,\
      ";\n Tipo da coluna odometro :", carros_1["odometer_km"].dtype )

Tipo da coluna preço : int64 ;
 Tipo da coluna odometro : int64


#### Converter a coluna `date_crawled`

In [42]:
%pip install datetime
import datetime as dt
def formatDate(dataset,column):
    dateformato = []
    for element in dataset[column]:
        element = dt.datetime.strptime(element, "%Y-%m-%d %H:%M:%S")
        element = element.date()
        dateformato.append(element)
    dataset[column] = dateformato         

You should consider upgrading via the '/dataquest/system/env/python3/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [40]:
formatDate(carros_1,'date_crawled')

In [43]:
n_dcrawled = carros_1['date_crawled'].value_counts(normalize=True, dropna = False).sort_index(ascending=True)*100
print(n_dcrawled )

2016-03-05    2.538
2016-03-06    1.394
2016-03-07    3.596
2016-03-08    3.330
2016-03-09    3.322
2016-03-10    3.212
2016-03-11    3.248
2016-03-12    3.678
2016-03-13    1.556
2016-03-14    3.662
2016-03-15    3.398
2016-03-16    2.950
2016-03-17    3.152
2016-03-18    1.306
2016-03-19    3.490
2016-03-20    3.782
2016-03-21    3.752
2016-03-22    3.294
2016-03-23    3.238
2016-03-24    2.910
2016-03-25    3.174
2016-03-26    3.248
2016-03-27    3.104
2016-03-28    3.484
2016-03-29    3.418
2016-03-30    3.362
2016-03-31    3.192
2016-04-01    3.380
2016-04-02    3.540
2016-04-03    3.868
2016-04-04    3.652
2016-04-05    1.310
2016-04-06    0.318
2016-04-07    0.142
Name: date_crawled, dtype: float64


In [44]:
periodo = n_dcrawled.index.max() - n_dcrawled.index.min()
print ("A data mais recente é", n_dcrawled.index.min(), ", a última é", n_dcrawled.index.max(),\
       ".\n Os dados contém informações de cerca de mês - ", periodo)

A data mais recente é 2016-03-05 , a última é 2016-04-07 .
 Os dados contém informações de cerca de mês -  33 days, 0:00:00


In [45]:
carros_1['date_crawled'].describe()

count          50000
unique            34
top       2016-04-03
freq            1934
Name: date_crawled, dtype: object

In [46]:
print("Na coluna `date_crawled` há", len(carros_1['date_crawled'].unique()),\
      "dados únicos para", len(carros_1) ,\
      "colunas.\n No dia 03 de Abril 2016 foram feitas a maior quantidade de registros")

Na coluna `date_crawled` há 34 dados únicos para 50000 colunas.
 No dia 03 de Abril 2016 foram feitas a maior quantidade de registros


#### Converter a coluna `ad_created`

In [47]:
formatDate(carros_1,'ad_created')

In [48]:
n_ad_created = carros_1['ad_created'].value_counts(normalize=True, dropna = False).sort_index(ascending=True)*100
print(n_ad_created)

2015-06-11    0.002
2015-08-10    0.002
2015-09-09    0.002
2015-11-10    0.002
2015-12-05    0.002
              ...  
2016-04-03    3.892
2016-04-04    3.688
2016-04-05    1.184
2016-04-06    0.326
2016-04-07    0.128
Name: ad_created, Length: 76, dtype: float64


In [49]:
periodo_ad = n_ad_created.index.max() - n_ad_created.index.min()
print("Existem", len(carros_1['ad_created'].unique()), "valores únicos em 'ad_created'.\n A primeira data é"\
, n_ad_created.index.min(),", a última é",n_ad_created.index.max(),".\n Assim, os dados contêm registros de", periodo_ad) 

Existem 76 valores únicos em 'ad_created'.
 A primeira data é 2015-06-11 , a última é 2016-04-07 .
 Assim, os dados contêm registros de 301 days, 0:00:00


In [50]:
carros_1['ad_created'].describe()

count          50000
unique            76
top       2016-04-03
freq            1946
Name: ad_created, dtype: object

 Vemos que no dia 03 de abril de 2016 houve a maior quantidade de registros (1946 linhas).

#### Converter a coluna `last_seen`

In [51]:
formatDate(carros_1,'last_seen')

In [52]:
n_last_seen = carros_1['last_seen'].value_counts(normalize=True, dropna = False).sort_index(ascending=True)*100
print(n_last_seen)

2016-03-05     0.108
2016-03-06     0.442
2016-03-07     0.536
2016-03-08     0.760
2016-03-09     0.986
2016-03-10     1.076
2016-03-11     1.252
2016-03-12     2.382
2016-03-13     0.898
2016-03-14     1.280
2016-03-15     1.588
2016-03-16     1.644
2016-03-17     2.792
2016-03-18     0.742
2016-03-19     1.574
2016-03-20     2.070
2016-03-21     2.074
2016-03-22     2.158
2016-03-23     1.858
2016-03-24     1.956
2016-03-25     1.920
2016-03-26     1.696
2016-03-27     1.602
2016-03-28     2.086
2016-03-29     2.234
2016-03-30     2.484
2016-03-31     2.384
2016-04-01     2.310
2016-04-02     2.490
2016-04-03     2.536
2016-04-04     2.462
2016-04-05    12.428
2016-04-06    22.100
2016-04-07    13.092
Name: last_seen, dtype: float64


In [53]:
carros_1['last_seen'].describe()

count          50000
unique            34
top       2016-04-06
freq           11050
Name: last_seen, dtype: object

In [54]:
periodo_ls = n_last_seen.index.max() - n_last_seen.index.min()
print("Existem", len(carros_1['last_seen'].unique()), "valores únicos na coluna 'last_seen'.\n A data mais antiga é"\
, n_last_seen.index.min(),", o mais recente é", n_last_seen.index.max(),".\n Assim, os dados contêm registros para", periodo_ls)

Existem 34 valores únicos na coluna 'last_seen'.
 A data mais antiga é 2016-03-05 , o mais recente é 2016-04-07 .
 Assim, os dados contêm registros para 33 days, 0:00:00


Vemos que no dia 06 de abril de 2016 houve a maior quantidade de registros (11.050 linhas).

#### Explorando a coluna `registration_year` 

 O tipo de dados é int64 (o método df.info()), então não é necessário converter os dados.

In [59]:
n_reg_ano = carros_1['registration_year'].value_counts(normalize=True, dropna = False).sort_index(ascending=True)*100
print(n_reg_ano)

1000    0.002
1001    0.002
1111    0.002
1500    0.002
1800    0.004
        ...  
6200    0.002
8888    0.002
9000    0.004
9996    0.002
9999    0.008
Name: registration_year, Length: 97, dtype: float64


O registro do carro começou em 1899 em Munique. E o ano de registro não pode ser posterior ao anúncio. Os dados nesta coluna devem ser limpos.

In [60]:
carros_1['registration_year'].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [61]:
periodo_reg = n_reg_ano.index.max() - n_reg_ano.index.min()
print("Existem", len(carros_1['registration_year'].unique()), "valores únicos em `registration_year`.\n A data mais antiga é"\
, n_reg_ano.index.min(),", a mais recente é", n_reg_ano.index.max(),".\n Assim, os dados contêm registros de", periodo_reg, "years")

Existem 97 valores únicos em `registration_year`.
 A data mais antiga é 1000 , a mais recente é 9999 .
 Assim, os dados contêm registros de 8999 years


#### Removendo dados imprecisos da coluna `registration_year`

Tomaremos 1900 como o limite inferior. Acima, descobrimos qual é a última data de criação dos registros no eBay, 07 de abril de 2016.

Tomaremos 2015 como o limite superior.

In [62]:
# removendo dados imprecisos 
carros_1 = carros_1[carros_1['registration_year'].between(1900,2015)]

In [63]:
n_reg_ano = carros_1['registration_year'].value_counts(normalize=True, dropna = False).sort_index(ascending=True)*100
print(n_reg_ano)

1910    0.019267
1927    0.002141
1929    0.002141
1931    0.002141
1934    0.004282
          ...   
2011    3.498030
2012    2.832249
2013    1.725467
2014    1.425758
2015    0.854170
Name: registration_year, Length: 77, dtype: float64


In [64]:
carros_1['registration_year'].describe()

count    46712.000000
mean      2002.431731
std          7.064689
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2007.000000
max       2015.000000
Name: registration_year, dtype: float64

In [65]:
print("Agora temos", len(carros_1), "colunas.")

Agora temos 46712 colunas.


In [66]:
carros_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46712 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date_crawled       46712 non-null  object
 1   name               46712 non-null  object
 2   price              46712 non-null  object
 3   ab_test            46712 non-null  object
 4   vehicle_type       46712 non-null  object
 5   registration_year  46712 non-null  int64 
 6   gearbox            46712 non-null  object
 7   power_ps           46712 non-null  int64 
 8   model              46712 non-null  object
 9   odometer           46712 non-null  object
 10  fuel_type          46712 non-null  object
 11  brand              46712 non-null  object
 12  unrepaired_damage  46712 non-null  object
 13  ad_created         46712 non-null  object
 14  last_seen          46712 non-null  object
 15  price_$            46712 non-null  int64 
 16  odometer_km        46712 non-null  int64

In [68]:
print("Os dados agora contém", len(carros_1.columns), "colunas e", len(carros_1.index), "linhas" )

Os dados agora contém 17 colunas e 46712 linhas


Neste projeto guiado do DataQuest (que por enquanto é parcialmente completo), aprendemos como explorar, descrever, limpar e modificar as informações em um determinado arquivo CSV. Também exploramos filtragem, agregação e visualização de dados como métodos de análise.